In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import data
from metrics import score

from preprocessing import Imputer, FeatureExtractor

KAGGLE = False
if KAGGLE:
    data.set_path('/kaggle/input/it-purple-hack-alpha/')

In [2]:
train = data.get_train_data()
test = data.get_test_data()

In [3]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, shuffle=False)
train.tail()

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
479995,159998,month_2,-0.044003,-0.057965,0.039715,-0.043190,channel_code_12,city_873,city_type_479,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,0.118568,0.255256,0.539375,{α},{α}
479996,159998,month_3,-0.027650,-0.038425,0.065658,-0.026786,channel_code_12,city_873,city_type_479,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,0.134735,0.255256,0.539375,{α},{α}
479997,159999,month_1,-0.154976,-0.203525,-0.125381,-0.154507,channel_code_9,city_99,city_type_0,index_city_code_91,...,0.944497,0.384773,-0.165131,0.547032,0.418798,-0.201123,0.250924,0.374540,{α},{α}
479998,159999,month_2,-0.156491,-0.204650,-0.125381,-0.156027,channel_code_9,city_99,city_type_0,index_city_code_91,...,0.944497,0.384773,-0.165131,0.547032,0.418798,-0.201123,0.250924,0.374540,{α},{α}
479999,159999,month_3,-0.156362,-0.204650,-0.124963,-0.155898,channel_code_9,city_99,city_type_0,index_city_code_91,...,0.944497,0.384773,-0.165131,0.547032,0.418798,-0.201123,0.250924,0.374540,{α},{α}


In [4]:
val.head()

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
480000,160000,month_1,-0.140437,-0.173369,-0.104965,-0.139923,channel_code_12,city_88,city_type_0,None,...,0.945281,0.407762,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{}
480001,160000,month_2,-0.148302,-0.198397,-0.106751,-0.147812,channel_code_12,city_88,city_type_0,None,...,0.944889,0.396267,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{}
480002,160000,month_3,-0.149538,-0.199147,-0.109430,-0.149052,channel_code_12,city_88,city_type_0,None,...,0.944889,0.396267,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{}
480003,160001,month_1,-0.151228,-0.185547,-0.125976,-0.150748,channel_code_0,city_6,city_type_0,None,...,0.946458,0.442244,-0.111110,0.550615,0.596576,-0.113140,0.254101,0.473441,{α},{α}
480004,160001,month_2,-0.149480,-0.178898,-0.125842,-0.148994,channel_code_0,city_6,city_type_0,None,...,0.946066,0.430750,-0.111074,0.550471,0.596576,-0.152004,0.253234,0.451463,{α},{α}


In [5]:
test.id.value_counts().value_counts()

count
3    90120
2     9880
Name: count, dtype: int64

In [6]:
# Add row with date=month_4 for those ids where it is missing (copy from month_5)
all_ids = test[test.date == 'month_5'].id.unique()
month4_ids = test[test.date == 'month_4'].id
missing_ids = list(set(all_ids) - set(month4_ids))

print(len(missing_ids))

month5_df = test.loc[test.date == 'month_5']
month5_df.date = 'month_4'
month5_df.start_cluster = '{}'
month5_df = month5_df[month5_df.id.isin(missing_ids)]
assert month5_df.shape[0] == len(missing_ids)

test = pd.concat([test, month5_df], axis=0).reset_index(drop=True).sort_values(by=['id', 'date'])
len(test)

9880


/tmp/ipykernel_443023/1198428122.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month5_df.date = 'month_4'
/tmp/ipykernel_443023/1198428122.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month5_df.start_cluster = '{}'


300000

In [24]:
train = train.convert_dtypes()
val = val.convert_dtypes()
test = test.convert_dtypes()

imputer = Imputer()
imputer.fit(train)

train = imputer.transform(train)
val = imputer.transform(val)
test = imputer.transform(test)

columns = train.columns.tolist()
columns.remove('id')
columns.remove('date')
columns.remove('end_cluster')

feature_extractor = FeatureExtractor(columns)
feature_extractor.fit(train)

X_train = feature_extractor.transform(train)
X_val = feature_extractor.transform(val)
X_test = feature_extractor.transform(test)

y_train = train['end_cluster']
y_val = val['end_cluster']

### Модель краткосрочного предсказания

In [8]:
def get_month_dataset(X):

    month1 = X.iloc[::3, :].copy().reset_index(drop=True)
    month2 = X.iloc[1::3, :].copy().reset_index(drop=True)
    month3 = X.iloc[2::3, :].copy().reset_index(drop=True)

    assert month1.shape[0] == month2.shape[0] == month3.shape[0]

    result = pd.concat([month1, month2, month3], axis=1)

    result.columns = [f'{col}_{i}' for i in range(1, 4) for col in X.columns]
    
    return result


month_X_train = get_month_dataset(X_train)
month_X_val = get_month_dataset(X_val)
month_X_test = get_month_dataset(X_test)

month_X_train, month_y_train = month_X_train.drop(columns=['start_cluster_3']), month_X_train['start_cluster_3']
month_X_val, month_y_val = month_X_val.drop(columns=['start_cluster_3']), month_X_val['start_cluster_3']

In [9]:
month_X_train.head()

,balance_amt_avg_1,balance_amt_max_1,balance_amt_min_1,balance_amt_day_avg_1,channel_code_1,city_1,city_type_1,index_city_code_1,ogrn_days_end_month_1,ogrn_days_end_quarter_1,...,cnt_days_deb_g_oper_3m_3,sum_cred_g_oper_3m_3,cnt_cred_g_oper_3m_3,cnt_days_cred_g_oper_3m_3,sum_deb_h_oper_3m_3,cnt_deb_h_oper_3m_3,cnt_days_deb_h_oper_3m_3,sum_cred_h_oper_3m_3,cnt_cred_h_oper_3m_3,cnt_days_cred_h_oper_3m_3
0,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,...,0.728507,0.033607,0.946458,0.442244,0.87705,0.551044,0.663243,0.810065,0.294829,0.956958
1,-0.081586,-0.09186,-0.11404,-0.08089,channel_code_2,city_14,city_type_0,index_city_code_46,0.324343,1.258747,...,0.772463,-0.019422,0.948027,0.488221,0.043221,0.560788,0.707687,-0.167905,0.259011,0.605309
2,-0.154685,-0.186795,-0.122805,-0.154215,channel_code_12,city_613,city_type_306,index_city_code_46,-0.256297,-1.257854,...,0.728507,-0.028584,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.176302,0.252368,0.429485
3,-0.156643,-0.204861,-0.12566,-0.156179,channel_code_14,city_21,city_type_0,index_city_code_46,-1.185321,-0.367365,...,0.728507,-0.028584,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.37454
4,-0.138847,-0.182486,-0.12563,-0.138328,channel_code_8,city_21,city_type_0,index_city_code_46,-1.417577,-0.444799,...,0.871364,0.053048,0.957051,0.637647,-0.078297,0.556919,0.64102,-0.172533,0.253523,0.462452


In [10]:
import gc
gc.collect()

0

Для начала попробуем обычную матрицу веротяностей

In [11]:
class DummyPredictor:
    def fit(self, X, y):
        pass
    
    def predict(self, X):
        return pd.get_dummies(X['start_cluster_2'])

    
class MonthProbabilityEstimator:
    def fit(self, X, y):
        X = X.copy()
        X = X[['start_cluster_1', 'start_cluster_2']]
        X['start_cluster'] = X['start_cluster_1'].astype(str) + '_' + X['start_cluster_2'].astype(str)
        X = X.drop(columns=['start_cluster_1', 'start_cluster_2'])

        self.probs = pd.crosstab(X['start_cluster'], y, normalize='index')
    
    def predict(self, X):
        X = X.copy()
        X = X[['start_cluster_1', 'start_cluster_2']]
        X['start_cluster'] = X['start_cluster_1'].astype(str) + '_' + X['start_cluster_2'].astype(str)
        X = X.drop(columns=['start_cluster_1', 'start_cluster_2'])

        for cluster2 in X['start_cluster'].unique():
            if cluster2 not in self.probs.index:
                cluster = cluster2.split('_')[1]
                probs = np.zeros(len(data.clusters))
                probs[data.clusters.index(cluster)] = 1
                self.probs = pd.concat([self.probs, pd.DataFrame([probs], columns=self.probs.columns, index=[cluster2])])

        return self.probs.loc[X['start_cluster']].values

dummy_predictor = DummyPredictor()
dummy_predictor.fit(month_X_train, month_y_train)

probability_estimator = MonthProbabilityEstimator()
probability_estimator.fit(month_X_train, month_y_train)

In [12]:
probability_estimator.probs

start_cluster_3,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
start_cluster,,,,,,,,,,,,,,,,,
{other}_{other},0.918806,0.000538,0.005646,0.035757,0.004302,0.000672,0.001748,0.000403,0.006318,0.010889,0.004705,0.000403,0.001613,0.000000,0.004302,0.003898,0.000000
{other}_{},0.000000,0.846154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000
"{other}_{α, β}",0.271186,0.000000,0.627119,0.033898,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050847,0.000000
"{other}_{α, γ}",0.120968,0.000000,0.000000,0.850806,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028226,0.000000
"{other}_{α, δ}",0.120000,0.000000,0.000000,0.000000,0.720000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.160000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
{α}_{α},0.000714,0.022301,0.003799,0.004897,0.003020,0.000011,0.000022,0.000022,0.001559,0.004403,0.000648,0.000220,0.000285,0.000011,0.000373,0.957714,0.000000
{λ}_{other},1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
{λ}_{},0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000


In [13]:
from sklearn.metrics import accuracy_score, roc_auc_score

month_y_val_pred = dummy_predictor.predict(month_X_val)
print('Dummy predictor score:', score(month_y_val, month_y_val_pred))

month_y_val_pred = probability_estimator.predict(month_X_val)
print('Probability estimator score:', score(month_y_val, month_y_val_pred))

Dummy predictor score: 0.9134134410143546
Probability estimator score: 0.9444630401101588


In [14]:
test_month6_estimation = probability_estimator.predict(month_X_test)

### Долгосрочное предсказание

In [51]:
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
from catboost import CatBoostClassifier

class YearProbabilityEstimator:
    def fit(self, X, y, X_val, y_val):
        start_cluster = X['start_cluster']
        start_cluster_val = X_val['start_cluster']
        X = X.copy().drop(columns=['start_cluster'])

        X_val = X_val.copy().drop(columns=['start_cluster'])
        X_val = X_val[start_cluster_val.isin(start_cluster.unique())]
        y_val = y_val[start_cluster_val.isin(start_cluster.unique())]
        
        self.models = {}
        for cluster in tqdm(data.clusters):

            model = CatBoostClassifier(cat_features=X.select_dtypes('string').columns.tolist(), verbose=1 if not KAGGLE else 0,
                                       loss_function='MultiClass', eval_metric='AUC', task_type='GPU' if KAGGLE else 'CPU',
                                       iterations=1 if not KAGGLE else None)

            model.fit(X[start_cluster == cluster], y[start_cluster == cluster], verbose=1 if not KAGGLE else 100,
                        eval_set=(X_val[start_cluster_val == cluster], y_val[start_cluster_val == cluster]))

            self.models[cluster] = model
    
    def predict(self, X):
        result = pd.DataFrame(index=X.index, columns=data.clusters)
        start_cluster = X['start_cluster']
        X = X.copy().drop(columns=['start_cluster'])

        for cluster, model in self.models.items():
            if cluster not in start_cluster.unique():
                continue
            result.loc[start_cluster == cluster, model.classes_] = model.predict_proba(X[start_cluster == cluster])

        return result.fillna(0).values

In [52]:
probability_estimator = YearProbabilityEstimator()
probability_estimator.fit(X_train, y_train, X_val, y_val)

  0%|          | 0/17 [00:00<?, ?it/s]

Learning rate set to 0.5


  6%|▌         | 1/17 [00:00<00:10,  1.48it/s]

0:	test: 0.4375000	best: 0.4375000 (0)	total: 291ms	remaining: 0us

bestTest = 0.4375
bestIteration = 0

Learning rate set to 0.5


 12%|█▏        | 2/17 [00:03<00:30,  2.06s/it]

0:	test: 0.5138460	best: 0.5138460 (0)	total: 2.4s	remaining: 0us

bestTest = 0.5138459715
bestIteration = 0



 12%|█▏        | 2/17 [00:03<00:29,  1.95s/it]


CatBoostError: /src/catboost/catboost/private/libs/algo/data.cpp:196: Dataset test #0 contains class label "{\xCE\xBB}" that is not present in the learn dataset

In [18]:
score(y_val, probability_estimator.predict(X_val).fillna(0).values)

/tmp/ipykernel_443023/1216411844.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  score(y_val, probability_estimator.predict(X_val).fillna(0).values)


0.8863868657208679

### Комбнация вероятностей

In [33]:
test_prob = np.zeros((100000, len(data.clusters), len(data.clusters)))
for i, cluster in tqdm(enumerate(data.clusters)):
    test_to_predict = X_test.iloc[2::3].copy()
    print(test_to_predict.shape)
    test_to_predict['start_cluster'] = cluster
    test_prob[:, i, :] = probability_estimator.predict(test_to_predict).fillna(0).values

0it [00:00, ?it/s]

(100000, 90)


/tmp/ipykernel_443023/763526777.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_prob[:, i, :] = probability_estimator.predict(test_to_predict).fillna(0).values
1it [00:02,  2.02s/it]

(100000, 90)


/tmp/ipykernel_443023/763526777.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_prob[:, i, :] = probability_estimator.predict(test_to_predict).fillna(0).values
2it [00:03,  1.98s/it]

...

/tmp/ipykernel_443023/763526777.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_prob[:, i, :] = probability_estimator.predict(test_to_predict).fillna(0).values
15it [00:25,  1.69s/it]

(100000, 90)


/tmp/ipykernel_443023/763526777.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_prob[:, i, :] = probability_estimator.predict(test_to_predict).fillna(0).values
16it [00:28,  1.88s/it]

(100000, 90)


/tmp/ipykernel_443023/763526777.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_prob[:, i, :] = probability_estimator.predict(test_to_predict).fillna(0).values
17it [00:29,  1.73s/it]


In [34]:
test_prob.shape

(100000, 17, 17)

In [35]:
import importlib
importlib.reload(data)

result = data.get_final_proba(test_month6_estimation, test_prob)
result.shape

(100000, 17)

In [39]:
sample_submission = data.get_sample_submission()
assert sample_submission.columns.tolist() == ['id'] + data.clusters

sample_submission.loc[:, data.clusters] = result

sample_submission.to_csv('submission.csv', index=False)

In [40]:
sample_submission.head()

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.025919,0.041902,0.024343,0.037118,0.009977,0.000425,0.001355,0.000243,0.011248,0.015523,0.009582,0.002182,0.004207,2.059077e-05,0.002921,0.813015,0.000020
1,200001,0.022041,0.521110,0.003967,0.006762,0.001394,0.000473,0.000437,0.000077,0.002262,0.014596,0.002195,0.000473,0.000940,2.723741e-05,0.000921,0.422142,0.000182
2,200002,0.706231,0.002861,0.014105,0.130342,0.008837,0.002658,0.006158,0.005691,0.016267,0.015862,0.015733,0.001959,0.004041,5.642580e-08,0.014279,0.054697,0.000278
3,200003,0.021622,0.545110,0.003809,0.006730,0.001317,0.000425,0.000451,0.000084,0.002217,0.014027,0.002312,0.000393,0.000981,3.329983e-05,0.000972,0.399355,0.000163
4,200004,0.039211,0.172581,0.021885,0.031812,0.003002,0.000693,0.000940,0.000075,0.006027,0.057391,0.007802,0.000522,0.006560,1.889260e-05,0.001789,0.649633,0.000058
